# Import all necessary libraries -

In [6]:
from splinter import Browser
from sqlalchemy import func, create_engine
from bs4 import BeautifulSoup as bs
import requests, re, time, pymysql, pandas as pd
pymysql.install_as_MySQLdb()

# Get location for scrape

In [7]:
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/atom.php'
reqs = requests.get(url)
print(reqs)

<Response [200]>


## Using Beautiful Soup to get what's on the page so that we can navigate

In [8]:
soup = bs(reqs.text, 'lxml')
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<title>ATOM Syndication</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/theme/site/earthquake/index.css" rel="stylesheet"/><link href="css/feedPages.css" rel="stylesheet"/><meta content="USGS Earthquake Hazards Program, responsible for monitoring, reporting, and researching earthquakes and earthquake hazards" name="description"/><meta content="aftershock,earthquake,epicenter,fault,foreshock,geologist,geophysics,hazard,hypocenter,intensity,intensity scale,magnitude,magnitude scale,mercalli,plate,richter,seismic,seismicity,seismogram,seismograph,seismologist,seismology,subduction,tectonics,tsunami,quake,sismologico,sismologia" name="keywords"/> <link href="https://fonts.googleapis.com/icon?family=Material+Icons|Merriweather:400,400italic,700|Source+Sans+Pro:400,300,700" rel="stylesheet"/>
</head>
<body>
<header class="site-header" role="banner">
<a class="site-logo" href="/" title="

## Take the links from the li and place them into empty list with .geojson instead of .atom at the end of each input.

In [9]:
listoflinks = []
for link in soup.find_all('li'):
    listoflinks.append(link.a['href'][:-4] + 'geojson')

In [10]:
url = url[:27]
for i in range(len(listoflinks)):
    listoflinks[i] = url + listoflinks[i]
print(listoflinks)

https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_hour.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_hour.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/1.0_hour.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_day.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_day.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/1.0_day.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_week.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_week.geojson
https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson
https:

# Make Database connection - Connect to Database 

In [11]:
remote_db_endpoint = ''
remote_db_port = ''
remote_db_user = ''
remote_db_pwd = ''
remote_db_name = ''

In [12]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Turn the list into a DataFrame

In [13]:
sqldf = pd.DataFrame({'geojsonLoc': listoflinks})
sqldf.count()

geojsonLoc    20
dtype: int64

## Make sure that the if_exists is what you want it to be. 

In [14]:
sqldf.to_sql('geojsonLoc', con=conn, if_exists='replace', index=False)

In [15]:
query = """SELECT * FROM geojsonLoc"""
implementCheck = pd.read_sql(query, con=conn)
implementCheck.count()

geojsonLoc    20
dtype: int64